# Cómo crear el modelo y su dataset para explicarlo usando EXPAI

Bienvenido/a a este primer producto de EXPAI para explicar los modelos analíticos de tu proceso de negocio. Para facilitar la incorporación de nuestros servicios en vuestros procesos de negocio, hemos preparado una demo explicando cómo generar los datos que necesita EXPAI.

Recuerda que en caso de encontraros con cualquier error o dificultad, podéis escribirnos a info@expai.io y la resolveremos lo antes posible.



## Creando el dataset que usaremos para crear el modelo de estimación de precio de venta de vehículos

En este caso de uso vamos a crear un modelo de estimación de precio de venta para vehículos de segunda mano. Para ello tenemos un dataset que nos ha pasado el equipo de producto cuyos campos son:

- car: marca del vehículo.
- price: precio de venta real.
- body: tipo de carrocería del vehículo.
- mileage: kilometraje, en millas, del vehículo.
- engV: tamaño del motor.
- engType: tipo de combustible.
- registration: indica si el vehículo estaba registrado.
- year: año del vehículo.
- model: modelo del vehículo.
- drive: tipo de tracción.

En EXPAI se pueden subir tres tipos de datasets, cuyo uso dependerá de cómo hayamos creado el modelo:

1. Si el modelo se ha creado usando Pipeline: en este caso la muestra que deberemos subir a EXPAI es la muestra original sin transformar, ya que el Pipeline que subiremos a EXPAI sabrá cómo transformarlo.

2. Si el modelo no se ha creado usando Pipeline: en este caso la muestra que deberemos subir a EXPAI pueden ser dos:

    - Muestra transformada usada para entrenar el modelo: deberemos subir a EXPAI el dato transformado que habremos usado para entrenar el modelo (conocido popularmente como X).
    - Muestra sin original sin transformar (muestra display): si se puede asegurar la existencia de una relación 1:1 entre las muestras transformadas y la original mediante el índice de las mismas, se puede subir la muestra original sin transformar a EXPAI (marcada como display) de tal manera que el sistema use el valor sin transformar para generar las gráficas de las explicaciones. Esto se hace con el objetivo de plantear gráficas userfriendly y entendibles por los usuarios no técnicos.

In [44]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn import model_selection, preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn import metrics as ms
import pickle as pickle
import os

**Cargamos el dato**

El dataset que nos han pasado es un fichero .csv, por lo que para leerlo usaremos el método _read_csv_ de Pandas.

In [47]:
# Define the path to the sample file
original_sample_path = os.path.abspath("/Users/danielhormigoruiz/Projects/jupyter/data/expai/demo_dag_dto/car_ad_display.csv")

# Read the file
df = pd.read_csv(original_sample_path, encoding='iso-8859-1', sep = ";", index_col=0)
df.head()

,car,price,body,mileage,engV,engType,registration,year,model,drive
0,Ford,15500.0,crossover,68,2.5,Gas,yes,2010,Kuga,full
1,Mercedes-Benz,20500.0,sedan,173,1.8,Gas,yes,2011,E-Class,rear
2,Mercedes-Benz,35000.0,other,135,5.5,Petrol,yes,2008,CL 550,rear
3,Mercedes-Benz,17800.0,van,162,1.8,Diesel,yes,2012,B 180,front
5,Nissan,16600.0,crossover,83,2.0,Petrol,yes,2013,X-Trail,full


**Eliminamos aquellos con precio de venta negativo**

Dado que el Pipeline de Scikit Learn no permite eliminar registros de un dataset, si tuviésemos que hacerlo tendremos que hacerlo antes de construir el Pipeline.

In [48]:
df = df.drop(df[df.price <= 0 ].index)

**Eliminamos los engV a nulo y menores que 40**

In [49]:
df = df.dropna(how = "any", subset = ["engV"])
df = df.drop(df[df.engV > 40].index)

**Eliminamos cualquier nan**

In [50]:
df = df.dropna()

**Definimos los datasets de entrenamiento y test**

In [51]:
y_train = df["price"]
x_train = df.drop(["price"], axis=1)
data_train, data_test, label_train, label_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

In [52]:
df.to_csv(os.path.abspath("../../data/expai/demo_dag_dto/car_ad_display.csv"), sep = ";", encoding = "iso-8859-1", index = True)

### Creando un modelo usando Pipelines

Vamos a crear un modelo usando los Pipelines de Scikit Learn. Para ello, crearemos un Pipeline con los siguientes pasos:

- Codificar las variables categóricas
- Entrenar el modelo

In [53]:
# Definimos el Transformer de las variables categóricas
transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ["mileage", "engV", "year"]),
        ('cat', OrdinalEncoder(handle_unknown='ignore'), ["car", "body", "engType", "registration", "model", "drive"])
    ]
)

In [54]:
# Definimos los parámetros de entrenamiento del XGBoost
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'silent': 1
}
model = xgb.XGBRegressor(**xgb_params)

In [55]:
# Creamos el pipeline de entrenamiento/ejecución del modelo
clf = Pipeline(steps=[
    ('preprocessor', transformer),
    ('model', model)
])

# Entrenamos el modelo
clf.fit(X = data_train, y = label_train)

[22:31:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['mileage', 'engV', 'year']),
                                                 ('cat',
                                                  OrdinalEncoder(handle_unknown='ignore'),
                                                  ['car', 'body', 'engType',
                                                   'registration', 'model',
                                                   'drive'])])),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.7, eta=0.05,
                              eval_...
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.0500000007, max_delta_step

In [57]:
print(f"""
    MSE: {ms.mean_squared_error(label_test, clf.predict(data_test))}\n
    R2: {ms.r2_score(label_test, clf.predict(data_test))}
""")


    MSE: 51754384.06679244

    R2: 0.9171054473853012



**Exportamos el modelo mediante Pickle**

Para poder subir el modelo a EXPAI es necesario exportar el pipeline usando Pickle, de manera genérica, o la función _save_ propia de la librería que estés usando.

In [56]:
# Guardamos el modelo en local, y luego lo subimos a EXPAI
model_path = os.path.abspath("../../data/expai/demo_dag_dto/model_pipeline.pkl")
with open(model_path, 'wb') as f:
    pickle.dump(clf, f)

### Creando un modelo sin Pipelines

En la siguiente sección veremos como crear un modelo sin usar Pipelines de manera que podamos subirlo a EXPAI. En este caso deberás subir a EXPAI dos cosas:

1. El dato transformado usado para entrenar/ejecutar el modelo.

2. El modelo

#### Creando el dataset

El objetivo es crear el dataset que deberás subir a EXPAI

**Codificamos las variables categóricas**

In [59]:
for c in df.columns:
    if df[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[c].values)) 
        df[c] = lbl.transform(list(df[c].values))

In [61]:
df.head()

,car,price,body,mileage,engV,engType,registration,year,model,drive
0,23,15500.0,0,68,2.5,1,1,2010,467,1
1,50,20500.0,3,173,1.8,1,1,2011,317,2
2,50,35000.0,2,135,5.5,3,1,2008,227,2
3,50,17800.0,5,162,1.8,0,1,2012,195,0
5,55,16600.0,0,83,2.0,3,1,2013,810,1


**Guardamos el dataset**

Siempre debemos indicar que hay una columna de índices (index = True), que será la que, en caso de querer lincar con su equivalente _display_, usaremos como clave para unir ambos datasets

In [ ]:
df.to_csv(os.path.abspath("../../data/expai/demo_dag_dto/sample.csv"), sep = ";", encoding = "utf-8", index = True)

#### Entrenamos el modelo

Y por último entrenamos el modelo de manera aislada

In [64]:
# Dividimos entre entrenamiento y test
y_train = df["price"]
x_train = df.drop(["price"], axis=1)
data_train, data_test, label_train, label_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

In [65]:
# Debido al tipo de modelo, es necesario convertirlo a un tipo de matriz especial
data_train_matrix = xgb.DMatrix(data_train, label_train)

# Entrenamos el modelo
cv_output = xgb.cv(xgb_params, data_train_matrix, num_boost_round=1000, early_stopping_rounds=20,
                   verbose_eval=50, show_stdv=False)
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), data_train_matrix, num_boost_round=num_boost_rounds)

[22:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip thr

In [66]:
# Predecimos 
y_predict = model.predict(xgb.DMatrix(data_test))

# Y calculamos el MSE y R2
print(f"""
    MSE: {ms.mean_squared_error(label_test, y_predict)}\n
    R2: {ms.r2_score(label_test, y_predict)}
""")


    MSE: 35488563.50026055

    R2: 0.9431582725340557



**Exportamos el modelo mediante Pickle**

Para poder subir el modelo a EXPAI es necesario exportar el pipeline usando Pickle, de manera genérica, o la función _save_ propia de la librería que estés usando.

In [9]:
model_path = os.path.abspath("../../data/expai/demo_dag_dto/model.pkl")
with open(model_path, 'wb') as f:
    pickle.dump(model, f)